In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import root_pandas as rp
import pandas as pd
from tqdm import tqdm
import lmfit

from scipy.optimize import curve_fit

In [ ]:
def func(x, a, b, c):
    """
    a => sigma_l + sigma_t
    b => epsilon*sigma_tt
    c => Sqrt(2epsilon(1+epsilon))* sigma_lt
    """
    f = a + b * np.cos(2*x) + c * np.cos(x)
    return f

In [ ]:
df = pd.read_csv("cross_section.txt")
df.head()

In [ ]:
def print_and_plot_cross_sections(df):    
    #fig = plt.figure(figsize=(16,9))
    x = np.linspace(0, 2*np.pi, num=500)
    for w in tqdm(np.unique(df.w)):
        for q2 in np.sort(np.unique(df.q2)):
            for theta in np.unique(df.cos_theta):
                # print(w,q2,theta)
                d = df[(df.w == w) & (df.q2 == q2) & (df.cos_theta == theta)]
                if len(d) >= 5:
                    fig = plt.figure(figsize=(16,9))
                    #fig = plt.figure(figsize=(12,9))
                    plt.title(f"Bin\nW: {w}\nQ2: {q2}\nCos(Theta): {theta}")
                    plt.xlim(0, 2*np.pi)
                    p0 = [np.mean(d.y), 1.0, 1.0]
                    plt.errorbar(d.phi, d.y, d.yerr, fmt='o', alpha=0.5)
                    try:
                        popt, pcov = curve_fit(func, d.phi, d.y, p0=p0, maxfev=800)
                        plt.plot(x, func(x, *popt), 'g--')
                    except RuntimeError:
                        pass
                    except TypeError:
                        pass
                    
                    plt.savefig(f'W_{w}_Q2_{q2}_cosT_{theta}.png')
                    
                    
                    
                
        

In [ ]:
#print_and_plot_cross_sections(df)

In [ ]:
def mcmc_plot_cross_sections(df):    
    x = np.linspace(0, 2*np.pi, num=500)
    for w in tqdm(np.unique(df.w)):
        for q2 in np.sort(np.unique(df.q2)):
            for theta in np.unique(df.cos_theta):
                d = df[(df.w == w) & (df.q2 == q2) & (df.cos_theta == theta)]
                if len(d) >= 5:
                    fig = plt.figure(figsize=(16,9))
                    plt.title(f"Bin\nW: {w}\nQ2: {q2}\nCos(Theta): {theta}")
                    plt.xlim(0, 2*np.pi)
                    p0 = [np.mean(d.y), 1.0, 1.0]
                    plt.errorbar(d.phi, d.y, d.yerr, fmt='o', alpha=0.5)
                    try:
                        
                        popt, pcov = curve_fit(func, d.phi, d.y, p0=p0, maxfev=800)
                        a = popt[0]
                        b = popt[1]
                        c = popt[2]
                        plt.plot(x, func(x, *popt), 'g--', 
                                 label=f'curve_fit: a = {a:0.3f}, b = {b:0.3f}, c = {c:0.3f}')
                        
                        
#                         model = lmfit.Model(func)
#                         p = model.make_params(a=a, b=b, c=c)
                        
#                         emcee_kws = dict(steps=1000, burn=300, thin=600, is_weighted=False,
#                                      progress=False)
#                         result_mcmc = model.fit(data=d.y, params=p, x=d.phi, method='emcee', 
#                                           nan_policy='omit',fit_kws=emcee_kws)
                        
#                         a = result_mcmc.params['a'].value
#                         b = result_mcmc.params['b'].value
#                         c = result_mcmc.params['c'].value
#                         plt.plot(x, model.eval(params=result_mcmc.params, x=x), 'r--', 
#                                 label=f'mcmc: a = {a:0.3f}, b = {b:0.3f}, c = {c:0.3f}')
                        
                    except RuntimeError:
                        print("RuntimeError")
                        pass
                    except TypeError:
                        print("TypeError")
                        pass
                    plt.legend()
                    plt.savefig(f'plots/mcmc_W_{w:0.3f}_Q2_{q2:0.3f}_cosT_{theta:0.3f}.png')
                    #plt.close()
                    

In [ ]:
mcmc_plot_cross_sections(df)

In [ ]:
fig = plt.figure(figsize=(16,9))
plt.hist2d(df.w, df.q2, bins=(len(np.unique(df.w)),len(np.unique(df.q2))))
plt.colorbar()
plt.show()